In [ ]:
%run translation.py

In [ ]:
import graphviz
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Image
import os
import tempfile


In [ ]:
def create_complex_example_model():
    """Create a more complex SPIN model demonstrating all transition types"""
    model = SPINtoPRISM()
    
    # Add places representing a manufacturing process
    model.add_place("start", 1, is_initial=True)
    model.add_place("cutting", 2)
    model.add_place("after_cut", 1)
    model.add_place("bending", 2)
    model.add_place("milling", 3)
    model.add_place("heavy_polish", 2)
    model.add_place("light_polish", 1)
    model.add_place("fine_depos", 1)
    model.add_place("rough_depos", 1)
    model.add_place("polish_done", 1)
    model.add_place("depos_done", 1)
    model.add_place("ready_paint", 1)  # NEW: intermediate place
    model.add_place("hphs_paint", 2)
    model.add_place("lpls_paint", 3)
    model.add_place("end", 1)
    
    # Add transitions
    # Initial task: cutting
    model.add_transition("t_cutting", TransitionType.TASK, 
                        ["start"], ["cutting"], 
                        impact_vector=[10, 1])
    
    # Task completion
    model.add_transition("t_cut_done", TransitionType.SINGLE,
                        ["cutting"], ["after_cut"])
    
    # Parallel split: bending and milling happen in parallel
    model.add_transition("t_parallel_split", TransitionType.PARALLEL_SPLIT,
                        ["after_cut"], ["bending", "milling"])
    
    # Nature decision after bending (quality check)
    model.add_transition("t_quality_check", TransitionType.NATURE,
                        ["bending"], ["heavy_polish", "light_polish"],
                        probability=0.2)  # 20% chance needs heavy polishing
    
    # Choice after milling (deposition method)
    model.add_transition("t_depos_choice", TransitionType.CHOICE,
                        ["milling"], ["fine_depos", "rough_depos"])
    
    # Task transitions for polishing
    model.add_transition("t_heavy_pol", TransitionType.TASK,
                        ["heavy_polish"], ["polish_done"],
                        impact_vector=[5, 4])
    
    model.add_transition("t_light_pol", TransitionType.TASK,
                        ["light_polish"], ["polish_done"],
                        impact_vector=[8, 1])
    
    # Task transitions for deposition  
    model.add_transition("t_fine_dep", TransitionType.TASK,
                        ["fine_depos"], ["depos_done"],
                        impact_vector=[30, 1])
    
    model.add_transition("t_rough_dep", TransitionType.TASK,
                        ["rough_depos"], ["depos_done"],
                        impact_vector=[10, 1])
    
    # Parallel merge: wait for both polishing and deposition to complete
    model.add_transition("t_parallel_merge", TransitionType.PARALLEL_MERGE,
                        ["polish_done", "depos_done"], ["ready_paint"])
    
    # Final choice: painting method (FIXED)
    model.add_transition("t_paint_choice", TransitionType.CHOICE,
                        ["ready_paint"], ["hphs_paint", "lpls_paint"])
    
    # Final tasks
    model.add_transition("t_hphs", TransitionType.TASK,
                        ["hphs_paint"], ["end"],
                        impact_vector=[40, 1])
    
    model.add_transition("t_lpls", TransitionType.TASK,
                        ["lpls_paint"], ["end"],
                        impact_vector=[20, 3])
    
    return model

In [ ]:

def visualize_and_display(model):
    """Create, render and display the SPIN visualization"""
    
    # Create the graphviz object with better formatting
    dot = graphviz.Digraph(comment='SPIN Model', format='png')
    dot.attr(rankdir='LR', size='12,8', dpi='300')
    dot.attr('node', fontsize='10', fontname='Arial')
    dot.attr('edge', fontsize='8', fontname='Arial')
    
    # Add places with better styling
    for name, place in model.places.items():
        if place.is_initial:
            shape = 'doublecircle'
            color = 'lightblue'
            style = 'filled'
        else:
            shape = 'circle'
            color = 'lightgray'
            style = 'filled'
            
        label = f"{name}\\nd={place.duration}"
        dot.node(name, label, shape=shape, color=color, style=style)
        
    # Add transitions and connections with better styling
    for transition in model.transitions:
        # Create transition node with type-specific styling
        if transition.type == TransitionType.NATURE:
            label = f"{transition.name}\\np={transition.probability}"
            color = 'lightcoral'
            style = 'filled'
        elif transition.type == TransitionType.CHOICE:
            label = f"{transition.name}\\n(choice)"
            color = 'lightblue'
            style = 'filled'
        elif transition.type == TransitionType.TASK:
            label = f"{transition.name}\\n{transition.impact_vector}"
            color = 'lightgreen'
            style = 'filled'
        elif transition.type == TransitionType.PARALLEL_SPLIT:
            label = f"{transition.name}\\n(||split)"
            color = 'lightyellow'
            style = 'filled'
        elif transition.type == TransitionType.PARALLEL_MERGE:
            label = f"{transition.name}\\n(||merge)"
            color = 'lightyellow'
            style = 'filled'
        else:
            label = transition.name
            color = 'white'
            style = 'filled'
            
        t_node = f"t_{transition.name}"
        dot.node(t_node, label, shape='box', color=color, style=style)
        
        # Add edges from input places to transition
        for p_in in transition.input_places:
            dot.edge(p_in, t_node, arrowsize='0.7')
            
        # Add edges from transition to output places
        for p_out in transition.output_places:
            dot.edge(t_node, p_out, arrowsize='0.7')
    
    # Render the graph
    try:
        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file = os.path.join(temp_dir, 'spin_model')
            
            # Render to file
            rendered_file = dot.render(temp_file, cleanup=True)
            
            # Display the image
            print("SPIN Model Visualization:")
            print("="*50)
            
            # Try to display in different ways depending on environment
            try:
                # For Jupyter notebooks
                from IPython.display import Image as IPImage, display
                display(IPImage(rendered_file))
            except ImportError:
                try:
                    # For regular Python with matplotlib
                    img = mpimg.imread(rendered_file)
                    plt.figure(figsize=(15, 10))
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title('SPIN Model Visualization', fontsize=16)
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"Could not display image with matplotlib: {e}")
                    print(f"Image saved to: {rendered_file}")
            
            print(f"\nGraph successfully rendered!")
            print(f"Nodes: {len(model.places)} places + {len(model.transitions)} transitions")
            print(f"Edges: {sum(len(t.input_places) + len(t.output_places) for t in model.transitions)}")
            
    except Exception as e:
        print(f"Error rendering graph: {e}")
        print("Falling back to DOT source code:")
        print(dot.source)
    
    return dot

# Create and visualize the model
print("Creating and visualizing SPIN model...")
model = create_complex_example_model()

# Print model summary
print("\n" + "="*60)
print("MODEL SUMMARY")
print("="*60)
model.print_model_summary()

# Visualize
print("\n" + "="*60)
print("VISUALIZATION")
print("="*60)
viz = visualize_and_display(model)

# Also show the DOT source for reference
print("\n" + "="*60)
print("DOT SOURCE CODE")
print("="*60)
print(viz.source)

In [ ]:
print(model.generate_prism_model())